In [1]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
# load all necessary libraries
import pandas as pd
import numpy as np # linear algebra

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix as sk_confusion_matrix
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt


from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection  import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer


from gensim.models.word2vec import Word2Vec
from gensim import models
from gensim.models import KeyedVectors


from sklearn.linear_model import LogisticRegression
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D,Lambda,BatchNormalization,Activation
from keras.layers import Conv1D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
import keras.backend as K

import tensorflow as tf
from keras.preprocessing import sequence

from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot
from keras.layers import Input,MaxPool1D
from keras.models import Model

from gensim.models.word2vec import Word2Vec
from gensim import models
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
Using TensorFlow backend.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8",

In [2]:
## initialise the inbuilt Stemmer and the Lemmatizer
stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def preprocess(document, stem=True):
    'changes document to lower case, removes stopwords and lemmatizes/stems the remainder of the sentence'

    # change sentence to lower case
    document = document.lower()

    # tokenize into words
    words = word_tokenize(document)

    # remove stop words
    words = [word for word in words if word not in stopwords.words("english")]

    if stem:
        words = [stemmer.stem(word) for word in words]
    else:
        words = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words]

    # join words to make sentence
    document = " ".join(words)

    return document

In [3]:
seq_len = max_len = 300 #512
b_size = 128
n_chan = 10
f_size = 3 # filter size


In [4]:

data = pd.read_csv("./../deceptive-opinion.csv")
data = data.loc[:,['text','deceptive']]

# stem messages
#messages = [preprocess(message, stem=True) for message in data.text]
data.text = data.text.apply(lambda message : preprocess(message, stem=False))
data['deceptive'] = data.deceptive.map({'truthful':1, 'deceptive':0})



word2vec_path = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
embeddings = KeyedVectors.load_word2vec_format(word2vec_path, binary=True)
    




In [5]:
docs_vectors = pd.DataFrame() # creating empty final dataframe
stopwords = nltk.corpus.stopwords.words('english') # removing stop words
for doc in data['text'].str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it
    temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
    for word in doc.split(' '): # looping through each word of a single document and spliting through space
        if word not in stopwords: # if word is not present in stopwords then (try)
            try:
                word_vec = embeddings[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                temp = temp.append(pd.Series(word_vec), ignore_index = True) # if word is present then append it to temporary dataframe
            except:
                pass
    doc_vector = temp.mean() # take the average of each column(w0, w1, w2,........w300)
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe
docs_vectors.shape

(1600, 300)

In [6]:
docs_vectors['deceptive'] = data['deceptive']
docs_vectors = docs_vectors.dropna()



df_train, df_test = train_test_split(docs_vectors, test_size = 0.2,
                                                   random_state = 1)

In [7]:
df_train.columns

Index([          0,           1,           2,           3,           4,
                 5,           6,           7,           8,           9,
       ...
               291,         292,         293,         294,         295,
               296,         297,         298,         299, 'deceptive'],
      dtype='object', length=301)

In [8]:
df_truthful = df_train.loc[df_train.deceptive == 1,:]
X_train = df_truthful.drop('deceptive', axis = 1)#df_truthful.text
X_test=df_test.drop('deceptive', axis = 1)#df_test.text
y_train= df_truthful.deceptive
y_test=df_test.deceptive

print('Training set size : ', (X_train.shape[0]))
print('Test set size : ', (X_test.shape[0]))

Training set size :  649
Test set size :  320


In [9]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
56,-0.036224,0.053467,0.001528,0.147537,-0.040525,-0.031574,0.050694,-0.086859,0.043079,0.108244,...,-0.036021,0.063228,-0.068679,0.031920,-0.013915,-0.052879,0.047099,-0.089803,0.017003,-0.019714
142,-0.020328,0.026736,-0.015993,0.107362,-0.020965,-0.009089,0.031549,-0.074745,0.117780,0.075880,...,-0.024337,0.058791,-0.083148,0.058323,0.007232,-0.047959,-0.018779,-0.065708,0.011070,-0.022916
863,-0.001726,0.032717,-0.007366,0.119849,-0.032654,0.014206,0.062951,-0.115829,0.112119,0.112033,...,-0.030691,0.105678,-0.071019,0.037070,0.005711,-0.060322,-0.005572,-0.014772,0.000807,-0.000735
940,0.022375,-0.003678,-0.014354,0.102914,-0.058834,0.007045,0.038389,-0.030491,0.106895,0.027724,...,-0.022563,0.076558,-0.035776,0.038398,0.002561,-0.037891,-0.007936,-0.036979,0.006770,-0.030956
325,0.012235,0.020841,-0.011218,0.092090,-0.033881,-0.044987,-0.004983,-0.065976,0.093136,0.105427,...,-0.058510,0.051776,-0.098013,0.026776,0.015542,-0.022785,0.014391,-0.058107,0.025673,0.011781


In [10]:
X_train.shape,X_test.shape

((649, 300), (320, 300))

In [11]:
vocab_size = X_train.shape[1]


In [12]:
print('Pad sequences (samples x time)')
x_train = X_train.values#sequence.pad_sequences(X_train_sparse.toarray(), maxlen=max_len)
x_test = X_test.values#sequence.pad_sequences(X_test_sparse.toarray(), maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (649, 300)
x_test shape: (320, 300)


In [13]:
x_train

array([[-0.03622437,  0.05346709,  0.00152823, ..., -0.08980326,
         0.01700299, -0.01971436],
       [-0.0203283 ,  0.02673594, -0.0159933 , ..., -0.06570764,
         0.0110704 , -0.02291571],
       [-0.00172588,  0.03271702, -0.00736564, ..., -0.0147716 ,
         0.00080654, -0.00073542],
       ...,
       [ 0.01907355,  0.04399586, -0.01094584, ..., -0.02640293,
         0.03587887, -0.01699666],
       [ 0.00832627,  0.03359188, -0.00959778, ...,  0.00704158,
         0.07156649, -0.00845684],
       [-0.0020501 ,  0.0452691 ,  0.00364821, ...,  0.00251923,
         0.01195448, -0.01465988]], dtype=float32)

In [14]:

def define_discriminator(in_shape=(max_len,1)):

    D = Sequential()
    D.add(Conv1D(n_chan,f_size,activation='relu',input_shape = (seq_len,1)))
    D.add(Flatten())
    D.add(Dense(1, activation='sigmoid'))
    D.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])
    return D




In [15]:
# define the standalone generator model
def define_generator(latent_dim):
    def Conv1DTranspose(inp,nf,ks,s=2,p='same'):
        x1 = Lambda(lambda x : K.expand_dims(x,axis=2))(inp)
        x2 = Conv2DTranspose(filters=nf,kernel_size=(ks,1),strides=(s,1),padding=p)(x1)
        return Lambda(lambda x :K.squeeze(x,axis=2))(x2)

    G = Sequential()
    G.add(Dense(int(seq_len*0.25)*n_chan,input_shape=(latent_dim,)))
    G.add(Reshape((int(seq_len*0.25),n_chan)))
    G.add(BatchNormalization(momentum= 0.8,epsilon=1.e-5))

    G.add(Lambda(lambda x : Conv1DTranspose(x,n_chan,f_size)))
    G.add(BatchNormalization(momentum= 0.8,epsilon=1.e-5))

    G.add(Lambda(lambda x : Conv1DTranspose(x,1,3)))
    G.add(Activation('sigmoid'))
    
    G.summary()
    return G


In [16]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [17]:
# load data
def load_real_samples():
    # load dataset
    (trainX, _), (_, _) = (x_train,y_train),(x_test,y_test)#load_data()

    return trainX

In [18]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # select data
    X = dataset[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return X, y

In [19]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = generator.predict(x_input)
    # create class labels
    y = zeros((n_samples, 1))
    return X, y

In [20]:
def plot_history(d1_hist, d2_hist, g_hist):
    # plot history
    pyplot.title("GAN+WORD2VEC")
    pyplot.plot(d1_hist, label='disc_real')
    pyplot.plot(d2_hist, label='disc_fake')
    pyplot.plot(g_hist, label='gen')
    pyplot.legend()
    pyplot.savefig('GAN_WORD2VEC_line_plot_loss.png')
    pyplot.close()

In [21]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    c1_hist, c2_hist, g_hist = list(), list(), list()
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
                 
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
            (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
            c1_hist.append(d_loss1)
            c2_hist.append(d_loss2)
            g_hist.append(g_loss)
            
            #print(X_real.shape,X_fake.shape)
            
    # save the generator model
    #g_model.save('generator.h5')
    #d_model.save('dis_generator.h5')
    plot_history(c1_hist, c2_hist, g_hist)
    return g_model,d_model

In [22]:
# size of the latent space
latent_dim = 100
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load data
dataset = load_real_samples()

dataset = np.reshape(dataset,(dataset.shape[0],dataset.shape[1],1))

# train model
model,d_model= train(generator, discriminator, gan_model, dataset, latent_dim)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 750)               75750     
_________________________________________________________________
reshape_1 (Reshape)          (None, 75, 10)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 75, 10)            40        
_________________________________________________________________
lambda_1 (Lambda)            (None, 150, 10)           0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 150, 10)           40        
_________________________________________________________________
lambda_6 (Lambda)            (None, 300, 1)            0         
_________________________________________________________________
activation_1 (Activation)    (None, 300, 1)            0         
Total para

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, 1/5, d1=0.698, d2=0.567 g=0.938
>1, 2/5, d1=0.683, d2=0.460 g=1.119
>1, 3/5, d1=0.674, d2=0.349 g=1.340
>1, 4/5, d1=0.666, d2=0.264 g=1.590
>1, 5/5, d1=0.657, d2=0.201 g=1.840
>2, 1/5, d1=0.645, d2=0.151 g=2.097
>2, 2/5, d1=0.631, d2=0.114 g=2.331
>2, 3/5, d1=0.615, d2=0.087 g=2.577
>2, 4/5, d1=0.598, d2=0.069 g=2.790
>2, 5/5, d1=0.573, d2=0.056 g=2.973
>3, 1/5, d1=0.548, d2=0.048 g=3.148
>3, 2/5, d1=0.522, d2=0.040 g=3.288
>3, 3/5, d1=0.492, d2=0.035 g=3.445
>3, 4/5, d1=0.459, d2=0.030 g=3.551
>3, 5/5, d1=0.425, d2=0.027 g=3.642
>4, 1/5, d1=0.391, d2=0.025 g=3.735
>4, 2/5, d1=0.358, d2=0.023 g=3.801
>4, 3/5, d1=0.323, d2=0.021 g=3.870
>4, 4/5, d1=0.289, d2=0.020 g=3.937
>4, 5/5, d1=0.256, d2=0.018 g=3.980
>5, 1/5, d1=0.229, d2=0.018 g=4.008
>5, 2/5, d1=0.200, d2=0.017 g=4.064
>5, 3/5, d1=0.177, d2=0.017 g=4.104
>5, 4/5, d1=0.152, d2=0.016 g=4.123
>5, 5/5, d1=0.133, d2=0.015 g=4.175
>6, 1/5, d1=0.115, d2=0.015 g=4.199
>6, 2/5, d1=0.100, d2=0.014 g=4.232
>6, 3/5, d1=0.085, d2=0.014 